In [ ]:
#CEll_1.
# -*- coding: utf-8 -*-
#CNN_GANDAM.ipynb

import os
import cv2
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix

In [ ]:
#CEll_2.
IMAGESIZE = 128  #読み込んだ画像は(IMAGESIZE,IMAGESIZE)のサイズにリサイズする

# ディレクトリ内の画像を読み込む
# inputpath: ディレクトリ文字列, imagesize: 画像サイズ, type_color: ColorかGray
def load_images(inputpath, imagesize, type_color):
    imglist = []
    filenamelist = []

    for root, dirs, files in os.walk(inputpath):
        for fn in sorted(files):
            bn, ext = os.path.splitext(fn)
            if ext not in [".bmp", ".BMP", ".jpg", ".JPG", ".jpeg", ".JPEG", ".png", ".PNG"]:
                continue

            filename = os.path.join(root, fn)

            if type_color == 'Color':
                # カラー画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_COLOR)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                testimage = np.asarray(testimage, dtype=np.float64)
                # 色チャンネル，高さ，幅に入れ替え．data_format="channels_first"を使うとき必要
                #testimage = testimage.transpose(2, 0, 1)
                # チャンネルをbgrの順からrgbの順に変更
                testimage = testimage[:,:,::-1]

            elif type_color == 'Gray':
                # グレースケール画像の場合
                testimage = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
                # サイズ変更
                height, width = testimage.shape[:2]
                testimage = cv2.resize(testimage, (imagesize, imagesize), interpolation = cv2.INTER_AREA)  #主に縮小するのでINTER_AREA使用
                # チャンネルの次元がないので1次元追加する
                testimage = np.asarray([testimage], dtype=np.float64)
                testimage = np.asarray(testimage, dtype=np.float64).reshape((imagesize, imagesize, 1))
                # チャンネル，高さ，幅に入れ替え．data_format="channels_first"を使うとき必要
                #testimage = testimage.transpose(2, 0, 1)

            imglist.append(testimage)
            filenamelist.append(fn)
    imgsdata = np.asarray(imglist, dtype=np.float32)

    return imgsdata, filenamelist  # 画像リストとファイル名のリストを返す


In [ ]:
#CEll_3.
### データ準備 ###
print('*** Loading images ***')

# 画像読み込みとラベル値作成
# クラス0
image0, filenames_image0 = load_images('./sundatabase_negative/', IMAGESIZE, 'Color')
label0 = np.full(image0.shape[0], 0)    #画像数と同数のラベル値を含むリスト

# クラス1
image1, filenames_image1 = load_images('./sundatabase_positive/', IMAGESIZE, 'Color')
label1 = np.full(image1.shape[0], 1)    #画像数と同数のラベル値を含むリスト


# 画像，ラベル値，ファイル名それぞれ1つの配列にまとめる
image_all = np.concatenate([image0, image1], axis=0)
label_all = np.append(label0, label1)
filename_all = filenames_image0 + filenames_image1


In [ ]:
#CEll_4.
indices = np.array(range(image_all.shape[0]))
image_train, image_test, label_train, label_test, index_train, index_test = train_test_split(image_all, label_all, indices, test_size=0.2)


In [ ]:
#Cell_5.
print(indices)
print(len(label_train))
print(len(label_test))

In [ ]:
#Cell_6.
# testデータのファイル名リスト作成
filenames_test = []
for i in range(len(index_test)):
    filenames_test.append(filename_all[index_test[i]])


# 画像の画素値を0-1に正規化
image_train /= 255.0
image_test /= 255.0

# ラベルをone hot vector形式に変換
label_train_binary = to_categorical(label_train)
label_test_binary = to_categorical(label_test)

print('Loaded images: ' + repr(image_train.shape[0]) + ' for training and ' + repr(image_test.shape[0]) + ' for testing.')


In [ ]:
#Cell_7.
### 画像分類モデル定義と学習処理の実行 ###
print('*** Training ***')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

# モデル定義（Sequential形式）
model = Sequential()

# 1st convolutional block
model.add(Conv2D(32, (3, 3), strides=1, padding='same',
                 activation='relu', input_shape=(IMAGESIZE, IMAGESIZE, 3)))
model.add(MaxPooling2D(pool_size=(4, 4), strides=4))

# 2nd convolutional block
model.add(Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=4))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(2, activation='softmax'))

# モデル構造を表示
print(model.summary())


In [ ]:
#Cell_8.
# traininigの設定
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# training実行
training = model.fit(image_train, label_train_binary, epochs=3, batch_size=2, verbose=1)

In [ ]:
#Cell_9.
### 学習済みモデルの評価 ###
print('*** Testing ***')

# testデータの分類精度を表示
scores = model.evaluate(image_test, label_test_binary, batch_size=5, verbose=0)
print("%s on test data: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#Confusion matrixを表示
results = model.predict(image_test, batch_size=5, verbose=1)
results_argmax = list(np.argmax(results, axis=-1))
cmatrix = confusion_matrix(label_test, results_argmax)
print('Confusion matrix:')
print(cmatrix)

In [ ]:
#Cell_10.
# testデータの分類結果をcsvファイルに書き出し
f = open('./result.csv', 'w')
writer = csv.writer(f, lineterminator='\n')

savedata = ['filename', 'true_class', 'estimate_class', 'estimate_likelihood_max', 'estimate_likelihood_class0', 'estimate_likelihood_class1']
writer.writerow(savedata)

for i in range(len(image_test)):
    savedata = [filenames_test[i], label_test[i], np.argmax(results[i]), np.max(results[i]), results[i][0], results[i][1]]
    writer.writerow(savedata)

f.close()

In [ ]:
inp = Input(shape=(IMAGESIZE, IMAGESIZE, 3))
_ = model(inp)

In [ ]:
#Cell_11.
target_layer = model.get_layer("conv2d_3")
intermediate_model = Model(inputs=[model.inputs], outputs=[target_layer.output, model.output])
label_test

In [ ]:
#Cell_12.
idx_candidates = np.where(label_test.ravel() == 1)[0]

if len(idx_candidates) == 0:
    raise ValueError("No sample with label == 1 found in label_test!")

idx = int(idx_candidates[0])   # first occurrence
print(idx)
print(f"Using sample index {idx} (label={label_test[idx]})")

# --- 2. Extract the corresponding image ---
input_img = image_test[idx]

# --- 3. Define intermediate model for Grad-CAM ---
target_layer = model.get_layer("conv2d_3")  # check the actual name in model.summary()

intermediate_model = Model(
    inputs=model.inputs,
    outputs=[target_layer.output, model.output]
)

import matplotlib.pyplot as plt
plt.imshow(input_img)

In [ ]:
#Cell_13.
# Suppose your Sequential model is named `model`
# Find the name of the last convolutional layer
import tensorflow as tf

model.summary()  # Check layer names (e.g., 'conv2d_1')

# Extract the output of the last conv layer
last_conv_layer = model.get_layer('conv2d_3')  # update name as needed

# Define an intermediate model that outputs both
intermediate_model = tf.keras.Model(
    inputs=model.inputs,
    outputs=[last_conv_layer.output, model.output]
)


In [ ]:
#Cell_14.
import tensorflow as tf
import numpy as np

# --- Grad-CAM 計算 ---
# 前処理済みの入力画像（NumPy）をTensorに変換
input_img_tensor = tf.convert_to_tensor(input_img.reshape(1, IMAGESIZE, IMAGESIZE, 3), dtype=tf.float32)

# 勾配を記録
with tf.GradientTape() as tape:
    # watch対象に指定
    tape.watch(input_img_tensor)

    # 特徴マップと最終出力を取得
    conv_output, predictions = intermediate_model(input_img_tensor, training=False)

    # 予測クラスのインデックスを決定
    class_idx = tf.argmax(predictions[0])

    # 目的クラスのスコア
    loss = predictions[:, class_idx]

# 勾配を計算（最後のConv層の出力に対する）
grads = tape.gradient(loss, conv_output)

# Global Average Pooling により重要度を計算
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

# 重み付き特徴マップの線形結合
conv_output = conv_output[0]  # shape: (H, W, Channels)
heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)

# ReLU + 正規化
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap) + 1e-8

# contrast stretching
vmin, vmax = np.percentile(heatmap, [1, 99])
heatmap = np.clip((heatmap - vmin) / (vmax - vmin + 1e-8), 0, 1)


In [ ]:
#Cell_15.
import cv2
import matplotlib.pyplot as plt
import numpy as np

# --- ensure normalization and contrast enhancement ---
heatmap = np.maximum(heatmap, 0)
if np.max(heatmap) > 0:
    vmin, vmax = np.percentile(heatmap, [1, 99])
    heatmap = np.clip((heatmap - vmin) / (vmax - vmin + 1e-8), 0, 1)
else:
    heatmap = np.zeros_like(heatmap)

# resize to match original image size
heatmap = cv2.resize(heatmap, (64, 64))
heatmap = np.power(heatmap, 0.7)  # smaller gamma → brighter

# convert to color map
heatmap_color = np.uint8(255 * heatmap)
heatmap_color = cv2.applyColorMap(heatmap_color, cv2.COLORMAP_JET)

# overlay
superimposed_img = cv2.addWeighted(input_img.astype('uint8'), 0.6, heatmap_color, 0.4, 0)

# show
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


In [ ]:
#Cell_16.
import cv2
import numpy as np
import matplotlib.pyplot as plt

def enhance_and_overlay(input_img, heatmap, output_size=(64, 64),
                        gamma=0.5, blend_ratio=(0.5, 0.8)):
    """
    Brightens and overlays a Grad-CAM heatmap on the original image.

    gamma < 1.0 → brighter mid-tones
    blend_ratio → (original_weight, heatmap_weight)
    """
    if hasattr(heatmap, "numpy"):
        heatmap = heatmap.numpy()
    heatmap = np.maximum(heatmap, 0)

    # Robust normalization
    vmin, vmax = np.percentile(heatmap, [1, 99])
    heatmap = np.clip((heatmap - vmin) / (vmax - vmin + 1e-8), 0, 1)

    # Gamma correction to brighten midrange
    heatmap = np.power(heatmap, gamma)

    # Resize and colorize
    heatmap_color = cv2.resize(heatmap, output_size)
    heatmap_color = np.uint8(255 * heatmap_color)
    heatmap_color = cv2.applyColorMap(heatmap_color, cv2.COLORMAP_JET)

    # Blend with original image
    orig_w, cam_w = blend_ratio
    superimposed = cv2.addWeighted(input_img.astype('uint8'),
                                   orig_w, heatmap_color, cam_w, 0)

    plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    return superimposed

# --- usage ---
_ = enhance_and_overlay(input_img, heatmap,
                        output_size=(64, 64), gamma=0.4)
